<a href="https://colab.research.google.com/github/RodrigoGuedesDP/Computer_Vision/blob/main/cv_labo3_vgg.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**Computer Vision -  Clasificación de expresiones faciales con VGG16**

---



*   En este laboratorio se utilizor el modelo VGG16 pre-entrenado para construir un clasificador con estos datos y evaluar su rendimiento. Para ello se utilizará un dataset público llamado "FER-2013".
**Autores:**  

Nieto Espinoza, Brajan E.  
[brajan.nieto@utec.edu.pe](mailto:brajan.nieto@utec.edu.pe)

Guedes del Pozo,  Rodrigo F.  
[rodrigo.guedes.d@utec.edu.pe](mailto:rodrigo.guedes.d@utec.edu.pe)

<img src="https://pregrado.utec.edu.pe/sites/default/files/logo-utec-h_0_0.svg" width="190" alt="Logo UTEC" loading="lazy" typeof="foaf:Image">      

---

# Laboratorio 03: Claisifcacion de Emociones con VGG16

In [ ]:

%pip install -q torch torchvision torchinfo scikit-learn matplotlib seaborn

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
import torchvision
from torchvision import datasets, transforms, models
from torchinfo import summary
import numpy as np
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns



Layer (type:depth-idx)                   Output Shape              Param #
VGG                                      [1, 1000]                 --
├─Sequential: 1-1                        [1, 512, 7, 7]            --
│    └─Conv2d: 2-1                       [1, 64, 224, 224]         1,792
│    └─ReLU: 2-2                         [1, 64, 224, 224]         --
│    └─Conv2d: 2-3                       [1, 64, 224, 224]         36,928
│    └─ReLU: 2-4                         [1, 64, 224, 224]         --
│    └─MaxPool2d: 2-5                    [1, 64, 112, 112]         --
│    └─Conv2d: 2-6                       [1, 128, 112, 112]        73,856
│    └─ReLU: 2-7                         [1, 128, 112, 112]        --
│    └─Conv2d: 2-8                       [1, 128, 112, 112]        147,584
│    └─ReLU: 2-9                         [1, 128, 112, 112]        --
│    └─MaxPool2d: 2-10                   [1, 128, 56, 56]          --
│    └─Conv2d: 2-11                      [1, 256, 56, 56]          29

## 2. Configuración de parámetros y dispositivo


In [ ]:
# Configuración
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Dispositivo utilizado: {device}')

# Parámetros
BATCH_SIZE = 32
NUM_EPOCHS = 5
NUM_CLASSES = 7  # 7 emociones: angry, disgust, fear, happy, neutral, sad, surprise
LEARNING_RATE = 0.001


## 3. Transformaciones para VGG16

VGG16 requiere imágenes de 224×224 píxeles normalizadas con valores específicos. Utilizamos las transformaciones estándar de ImageNet.


In [ ]:
# Transformaciones para entrenamiento: redimensionar, recortar y normalizar
train_transforms = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(p=0.5),  # Data augmentation
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Transformaciones para validación/prueba: solo redimensionar y normalizar
test_transforms = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])


Total de parámetros (sin pesos preentrenados): 138357544 ≈ 138.36 millones


## 4. Cargar el dataset con ImageFolder

El dataset FER-2013 ya está organizado en carpetas `train` y `test`, cada una con subcarpetas por emoción. Utilizamos `datasets.ImageFolder` para cargar los datos.

In [ ]:
# Cargar datasets
train_dataset = datasets.ImageFolder(root='train', transform=train_transforms)
test_dataset = datasets.ImageFolder(root='test', transform=test_transforms)

# Crear DataLoaders
# Nota: num_workers=0 en Windows para evitar problemas de multiprocessing
import os
num_workers = 0 if os.name == 'nt' else 2
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=num_workers)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=num_workers)

# Mostrar información del dataset
print(f'Número de clases: {len(train_dataset.classes)}')
print(f'Clases: {train_dataset.classes}')
print(f'Número de imágenes de entrenamiento: {len(train_dataset)}')
print(f'Número de imágenes de prueba: {len(test_dataset)}')
print(f'Tamaño del batch: {BATCH_SIZE}')



Layer (type:depth-idx)                   Output Shape              Param #
VGG                                      [1, 1000]                 --
├─Conv2d: 1-1                            [1, 64, 224, 224]         1,792
├─ReLU: 1-2                              [1, 64, 224, 224]         --
├─Conv2d: 1-3                            [1, 64, 224, 224]         36,928
├─ReLU: 1-4                              [1, 64, 224, 224]         --
├─MaxPool2d: 1-5                         [1, 64, 112, 112]         --
├─Conv2d: 1-6                            [1, 128, 112, 112]        73,856
├─ReLU: 1-7                              [1, 128, 112, 112]        --
├─Conv2d: 1-8                            [1, 128, 112, 112]        147,584
├─ReLU: 1-9                              [1, 128, 112, 112]        --
├─MaxPool2d: 1-10                        [1, 128, 56, 56]          --
├─Conv2d: 1-11                           [1, 256, 56, 56]          295,168
├─ReLU: 1-12                             [1, 256, 56, 56]       

## 5. Cargar VGG16 preentrenado y modificar la capa de salida

Cargamos VGG16 con pesos preentrenados en ImageNet y reemplazamos la última capa completamente conectada para que tenga 7 salidas (una por cada emoción).

In [ ]:
# Cargar VGG16 preentrenado
model = models.vgg16(weights='IMAGENET1K_V1')

# Reemplazar la capa de salida (classifier[-1] es la última capa Linear)
# VGG16 tiene 1000 clases por defecto, necesitamos 7
num_features = model.classifier[6].in_features
model.classifier[6] = nn.Linear(num_features, NUM_CLASSES)

# Mover el modelo al dispositivo
model = model.to(device)

print('Modelo VGG16 cargado y modificado')
print(f'Última capa reemplazada: {model.classifier[6]}')

Downloading: "https://download.pytorch.org/models/vgg16-397923af.pth" to /root/.cache/torch/hub/checkpoints/vgg16-397923af.pth


100%|██████████| 528M/528M [00:07<00:00, 77.2MB/s]


Layer (type:depth-idx)                   Output Shape              Param #
VGG                                      [1, 1000]                 --
├─Sequential: 1-1                        [1, 512, 7, 7]            --
│    └─Conv2d: 2-1                       [1, 64, 224, 224]         1,792
│    └─ReLU: 2-2                         [1, 64, 224, 224]         --
│    └─Conv2d: 2-3                       [1, 64, 224, 224]         36,928
│    └─ReLU: 2-4                         [1, 64, 224, 224]         --
│    └─MaxPool2d: 2-5                    [1, 64, 112, 112]         --
│    └─Conv2d: 2-6                       [1, 128, 112, 112]        73,856
│    └─ReLU: 2-7                         [1, 128, 112, 112]        --
│    └─Conv2d: 2-8                       [1, 128, 112, 112]        147,584
│    └─ReLU: 2-9                         [1, 128, 112, 112]        --
│    └─MaxPool2d: 2-10                   [1, 128, 56, 56]          --
│    └─Conv2d: 2-11                      [1, 256, 56, 56]          29

## 6. Congelar las capas convolucionales

Congelamos los parámetros de las capas convolucionales (features) para que no se actualicen durante el entrenamiento. Solo entrenaremos las capas completamente conectadas (classifier).


In [ ]:
# Congelar las capas convolucionales (features)
for param in model.features.parameters():
    param.requires_grad = False

# Descongelar las capas del clasificador
for param in model.classifier.parameters():
    param.requires_grad = True

# Verificar cuántos parámetros son entrenables
trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
total_params = sum(p.numel() for p in model.parameters())
print(f'Parámetros entrenables: {trainable_params:,} ({trainable_params/total_params*100:.2f}%)')
print(f'Total de parámetros: {total_params:,}')


## 7. Configurar función de pérdida y optimizador

In [ ]:
# Función de pérdida
criterion = nn.CrossEntropyLoss()

# Optimizador (solo para los parámetros entrenables)
optimizer = optim.Adam(model.classifier.parameters(), lr=LEARNING_RATE)

print(f'Función de pérdida: {criterion}')
print(f'Optimizador: {optimizer}')


Predicción: lorikeet


## 8. Entrenamiento del modelo

Entrenamos el modelo durante 5 épocas, registrando la pérdida y precisión en cada época.


In [ ]:
# Listas para almacenar métricas
train_losses = []
train_accuracies = []

# Entrenamiento
model.train()
for epoch in range(NUM_EPOCHS):
    running_loss = 0.0
    correct = 0
    total = 0

    for batch_idx, (images, labels) in enumerate(train_loader):
        images = images.to(device)
        labels = labels.to(device)

        # Forward pass
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)

        # Backward pass
        loss.backward()
        optimizer.step()

        # Estadísticas
        running_loss += loss.item()
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

        # Mostrar progreso cada 100 batches
        if (batch_idx + 1) % 100 == 0:
            print(f'Época [{epoch+1}/{NUM_EPOCHS}], Batch [{batch_idx+1}/{len(train_loader)}], '
                  f'Pérdida: {loss.item():.4f}, Precisión: {100*correct/total:.2f}%')

    # Métricas de la época
    epoch_loss = running_loss / len(train_loader)
    epoch_accuracy = 100 * correct / total
    train_losses.append(epoch_loss)
    train_accuracies.append(epoch_accuracy)

    print(f'Época [{epoch+1}/{NUM_EPOCHS}] completada - '
          f'Pérdida promedio: {epoch_loss:.4f}, Precisión: {epoch_accuracy:.2f}%')
    print('-' * 60)

print('Entrenamiento completado!')


## 9. Evaluación del modelo en el conjunto de prueba

Evaluamos el rendimiento del modelo en el conjunto de prueba y mostramos métricas detalladas.


In [ ]:
# Evaluación
model.eval()
all_preds = []
all_labels = []
test_loss = 0.0
correct = 0
total = 0

with torch.no_grad():
    for images, labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)

        outputs = model(images)
        loss = criterion(outputs, labels)
        test_loss += loss.item()

        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

        # Guardar predicciones y etiquetas para métricas detalladas
        all_preds.extend(predicted.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())

# Métricas generales
test_accuracy = 100 * correct / total
avg_test_loss = test_loss / len(test_loader)

print('=' * 60)
print('RESULTADOS EN EL CONJUNTO DE PRUEBA')
print('=' * 60)
print(f'Pérdida promedio: {avg_test_loss:.4f}')
print(f'Precisión: {test_accuracy:.2f}%')
print(f'Total de muestras: {total}')
print(f'Predicciones correctas: {correct}')
print('=' * 60)


## 10. Métricas detalladas por clase

Mostramos el reporte de clasificación y la matriz de confusión para analizar el rendimiento por cada emoción.

In [ ]:
# Reporte de clasificación
print('\nREPORTE DE CLASIFICACIÓN POR CLASE:')
print('=' * 60)
print(classification_report(all_labels, all_preds,
                          target_names=train_dataset.classes,
                          digits=4))
print('=' * 60)


In [ ]:
# Matriz de confusión
cm = confusion_matrix(all_labels, all_preds)

# Visualizar matriz de confusión
plt.figure(figsize=(10, 8))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
            xticklabels=train_dataset.classes,
            yticklabels=train_dataset.classes)
plt.title('Matriz de Confusión - Clasificación de Emociones')
plt.ylabel('Etiqueta Real')
plt.xlabel('Etiqueta Predicha')
plt.tight_layout()
plt.show()

In [ ]:
# Gráficas de entrenamiento
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 4))

# Gráfica de pérdida
ax1.plot(range(1, NUM_EPOCHS + 1), train_losses, 'b-o')
ax1.set_xlabel('Época')
ax1.set_ylabel('Pérdida')
ax1.set_title('Pérdida de Entrenamiento')
ax1.grid(True)

# Gráfica de precisión
ax2.plot(range(1, NUM_EPOCHS + 1), train_accuracies, 'r-o')
ax2.set_xlabel('Época')
ax2.set_ylabel('Precisión (%)')
ax2.set_title('Precisión de Entrenamiento')
ax2.grid(True)

plt.tight_layout()
plt.show()


## 12. Análisis de resultados

### Explicación de los resultados obtenidos:

1. **Precisión general**: El modelo alcanzó una precisión del X% en el conjunto de prueba, lo que indica...

2. **Rendimiento por clase**:
   - Las emociones con mejor rendimiento son...
   - Las emociones más difíciles de clasificar son...

3. **Matriz de confusión**: La matriz muestra que el modelo confunde principalmente...

4. **Transfer Learning**: Al usar VGG16 preentrenado y congelar las capas convolucionales, aprovechamos las características aprendidas en ImageNet, lo que permite un buen rendimiento incluso con pocas épocas de entrenamiento.

5. **Limitaciones**:
   - El dataset tiene desbalance de clases (disgust tiene menos muestras)
   - 5 épocas pueden ser insuficientes para un ajuste fino completo
   - Las imágenes son en escala de grises convertidas a RGB, lo que puede afectar el rendimiento
